<a href="https://colab.research.google.com/github/TharunSaiVT/INFO-5731/blob/main/V_T_Tharun_Sai_Exercise_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 In-class Exercise 4**

**This exercise will provide a valuable learning experience in working with text data and extracting features using various topic modeling algorithms. Key concepts such as Latent Dirichlet Allocation (LDA), Latent Semantic Analysis (LSA), lda2vec, and BERTopic.**

***Please use the text corpus you collected in your last in-class-exercise for this exercise. Perform the following tasks***.

**Expectations**:
*   Students are expected to complete the exercise during lecture period to meet the active participation criteria of the course.
*   Use the provided .*ipynb* document to write your code & respond to the questions. Avoid generating a new file.
*   Write complete answers and run all the cells before submission.
*   Make sure the submission is "clean"; *i.e.*, no unnecessary code cells.
*   Once finished, allow shared rights from top right corner (*see Canvas for details*).

**Total points**: 40

**Deadline**: This in-class exercise is due at the end of the day tomorrow, at 11:59 PM.

**Late submissions will have a penalty of 10% of the marks for each day of late submission, and no requests will be answered. Manage your time accordingly.**


## Question 1 (10 Points)

**Generate K topics by using LDA, the number of topics K should be decided by the coherence score, then summarize what are the topics.**

You may refer the code here: https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/

In [ ]:
# Write your code here
import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel
import spacy

# Sample data
sample_data = [
     "The movie was fantastic, I absolutely loved it!",
    "I found the film to be very disappointing and boring.",
    "This movie exceeded my expectations, it was amazing!",
    "The acting in this film was subpar and the plot was weak.",
    "I couldn't stop laughing, this movie is hilarious!"
]

# Step 1: Preprocess the text data
nlp = spacy.load("en_core_web_sm")

def preprocess_text(texts):
    processed_texts = []
    for text in texts:
        doc = nlp(text)
        tokens = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
        processed_texts.append(tokens)
    return processed_texts

processed_data = preprocess_text(sample_data)

# Step 2: Create a dictionary and document-term matrix
id2word = corpora.Dictionary(processed_data)
corpus = [id2word.doc2bow(text) for text in processed_data]

# Step 3: Determine the optimal number of topics (K) using coherence scores
coherence_scores = []
for k in range(2, 11):
    lda_model = gensim.models.LdaModel(corpus=corpus, id2word=id2word, num_topics=k, random_state=100, update_every=1, chunksize=100, passes=10, alpha='auto', per_word_topics=True)
    coherence_model = CoherenceModel(model=lda_model, texts=processed_data, dictionary=id2word, coherence='c_v')
    coherence_score = coherence_model.get_coherence()
    coherence_scores.append((k, coherence_score))

# Select the K with the highest coherence score
best_k, best_coherence = max(coherence_scores, key=lambda x: x[1])
print(f"Optimal number of topics (K): {best_k}")

# Step 5: Train the LDA model with the selected K
lda_model = gensim.models.LdaModel(corpus=corpus, id2word=id2word, num_topics=best_k, random_state=100, update_every=1, chunksize=100, passes=10, alpha='auto', per_word_topics=True)

# Step 6: Summarize the topics
topics = lda_model.print_topics(num_topics=best_k, num_words=5)
for topic in topics:
    print(f"Topic {topic[0] + 1}: {topic[1]}")

Optimal number of topics (K): 8
Topic 1: 0.056*"movie" + 0.056*"film" + 0.056*"love" + 0.056*"hilarious" + 0.056*"exceed"
Topic 2: 0.056*"movie" + 0.056*"film" + 0.056*"exceed" + 0.056*"love" + 0.056*"boring"
Topic 3: 0.056*"movie" + 0.056*"film" + 0.056*"boring" + 0.056*"exceed" + 0.056*"fantastic"
Topic 4: 0.180*"expectation" + 0.180*"amazing" + 0.180*"exceed" + 0.180*"movie" + 0.020*"film"
Topic 5: 0.180*"absolutely" + 0.180*"fantastic" + 0.180*"love" + 0.180*"movie" + 0.020*"film"
Topic 6: 0.056*"movie" + 0.056*"film" + 0.056*"exceed" + 0.056*"love" + 0.056*"boring"
Topic 7: 0.180*"disappointing" + 0.180*"find" + 0.180*"boring" + 0.180*"film" + 0.020*"movie"
Topic 8: 0.100*"movie" + 0.100*"film" + 0.100*"weak" + 0.100*"subpar" + 0.100*"plot"


## Question 2 (10 Points)

**Generate K topics by using LSA, the number of topics K should be decided by the coherence score, then summarize what are the topics.**

You may refer the code here: https://www.datacamp.com/community/tutorials/discovering-hidden-topics-python

In [ ]:
# Write your code here
# Write your code here
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import numpy as np

# Sample data
sample_data = [
     "The movie was fantastic, I absolutely loved it!",
    "I found the film to be very disappointing and boring.",
    "This movie exceeded my expectations, it was amazing!",
    "The acting in this film was subpar and the plot was weak.",
    "I couldn't stop laughing, this movie is hilarious!"
]

# Step 1: Preprocess the text data and create a TF-IDF matrix
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(sample_data)

# Step 2: Determine the optimal number of topics (K) using LSA (you specify K)
K = 2  # You can choose the number of topics
lsa = TruncatedSVD(n_components=K)
lsa_topic_matrix = lsa.fit_transform(tfidf_matrix)

# Step 3: Summarize the topics
terms = tfidf_vectorizer.get_feature_names_out()
topic_keywords = []
for i, topic in enumerate(lsa.components_):
    top_terms = [terms[idx] for idx in topic.argsort()[-5:][::-1]]
    topic_keywords.append(top_terms)
    print(f"Topic {i + 1}: {', '.join(top_terms)}")


Topic 1: was, the, movie, it, this
Topic 2: and, film, the, found, be


## Question 3 (10 points):
**Generate K topics by using lda2vec, the number of topics K should be decided by the coherence score, then summarize what are the topics.**

You may refer the code here: https://nbviewer.org/github/cemoody/lda2vec/blob/master/examples/twenty_newsgroups/lda2vec/lda2vec.ipynb

In [ ]:
# Write your code here


## Question 4 (10 points):
**Generate K topics by using BERTopic, the number of topics K should be decided by the coherence score, then summarize what are the topics.**

You may refer the code here: https://colab.research.google.com/drive/1FieRA9fLdkQEGDIMYl0I3MCjSUKVF8C-?usp=sharing

In [ ]:
# Write your code here


## Extra Question (5 Points)

**Compare the results generated by the four topic modeling algorithms, which one is better? You should explain the reasons in details.**

**This question will compensate for any points deducted in this exercise. Maximum marks for the exercise is 40 points.**

In [ ]:
# Write your code here


# Mandatory Question

**Important: Reflective Feedback on this exercise**

Please provide your thoughts and feedback on the exercises you completed in this assignment.

Consider the following points in your response:

**Learning Experience:** Describe your overall learning experience in working with text data and extracting features using various topic modeling algorithms. Did you understand these algorithms and did the implementations helped in grasping the nuances of feature extraction from text data.

**Challenges Encountered:** Were there specific difficulties in completing this exercise?

Relevance to Your Field of Study: How does this exercise relate to the field of NLP?

**(Your submission will not be graded if this question is left unanswered)**



In [ ]:
# Your answer here (no code for this question, write down your answer as detail as possible for the above questions):

'''
Please write you answer here:





'''